# Advent of Code 2024 - J06

In [1]:
def read_input(kind):
    assert kind in ('input', 'example'), '"kind" must be "input" or "example"'
    imap = []
    with open(kind) as f:
        for line in f:
            imap.append(list(line.strip()))
    l = len(imap[0])
    assert all(len(line) == l for line in imap), 'all lines should be the same length'
    return imap

In [2]:
emap = read_input("example")
imap = read_input("input")

## First part

In [3]:
def count_visited_positions(imap):
    s = 0
    for line in imap:
        for pos in line:
            if pos == 'X':
                s += 1
    return s

In [4]:
def find_position(imap):
    for y, line in enumerate(imap):
        for x, pos in enumerate(line):
            if pos in ['^', '>', 'v', '<']:
                return (y, x)
    raise Exception("Initial position not found ! There should be one of these : '^', '>', 'v', '<'")

In [5]:
find_position(emap)

(6, 4)

In [6]:
def go_out(imap, y, x):
    nb_lines = len(imap)
    nb_cols = len(imap[0])
    if y == 0 and imap[y][x] == '^':
        return True
    if y == nb_lines-1 and imap[y][x] == 'v':
        return True
    if x == 0 and imap[y][x] == '<':
        return True
    if x == nb_cols-1 and imap[y][x] == '>':
        return True
    return False

In [7]:
def visit(imap):
    direction = {
        '^': (-1, 0),
        'v': (1, 0),
        '<': (0, -1),
        '>': (0, 1)
    }
    next_direction = ['^', '>', 'v', '<', '^']
    y, x = find_position(imap)
    while not(go_out(imap, y, x)):
        going_to = imap[y][x]
        dir_y, dir_x = direction[going_to]
        if imap[y + dir_y][x + dir_x] == '#':
            going_to = next_direction[next_direction.index(going_to) + 1]
            dir_y, dir_x = direction[going_to]
        imap[y][x] = 'X'
        y += dir_y
        x += dir_x
        imap[y][x] = going_to
    imap[y][x] = 'X'
    return imap

In [8]:
def first_part(kind):
    assert kind in ('input', 'example'), '"kind" must be "input" or "example"'
    imap = read_input(kind)
    return count_visited_positions(visit(imap))

In [9]:
first_part("example")

41

In [10]:
first_part("input")

4988

## Second part

In [11]:
def print_map(imap):
    for line in imap:
        nl = ''
        for car in line:
            nl += car
        print(nl)

In [12]:
def looping(imap):
    # Doesn't work (yet?)
    direction = {
        '^': (-1, 0),
        'v': (1, 0),
        '<': (0, -1),
        '>': (0, 1)
    }
    next_direction = ['^', '>', 'v', '<', '^']
    y, x = find_position(imap)
    loop = False
    while not(go_out(imap, y, x)):
        # Find next position:
        going_to = imap[y][x]
        dir_y, dir_x = direction[going_to]
        
        # Check if we have already been there:
        if imap[y + dir_y][x + dir_x] in ['|', '-', '+']:
            # If yes, we need to check if we were in the same direction. If yes, we have a loop:
            if going_to in ['^', 'v'] and imap[y + dir_y][x + dir_x] == '|':
                return True
            if going_to in ['<', '>'] and imap[y + dir_y][x + dir_x] == '-':
                return True
        
        # If all previous tests failed, thats means that we never visited this position yet:
        assert imap[y][x] == '^', f"should be '.' in position ({y}, {x}) and not '{imap[y][x]}'"
        
        # Check if there is an obstacle:
        if imap[y + dir_y][x + dir_x] in ['#', 'O']:
            going_to = next_direction[next_direction.index(going_to) + 1]
            dir_y, dir_x = direction[going_to]
            imap[y][x] = '+'
        elif going_to in ['^', 'v']:
            imap[y][x] = '|'
        elif going_to in ['<', '>']:
            imap[y][x] = '-'
        else: # Should not exist
            raise Exception(f"We have a problem here… ({y}, {x})")
        
        y += dir_y
        x += dir_x
        imap[y][x] = going_to
    imap[y][x] = 'X'
    return False

In [13]:
def lazy_looping(imap):
    # Same function as in first part, but we will check the number of visited places and stop if it becomes too high
    max_nb_visited_places = len(imap) * len(imap[0])
    nb = 0
    direction = {
        '^': (-1, 0),
        'v': (1, 0),
        '<': (0, -1),
        '>': (0, 1)
    }
    next_direction = ['^', '>', 'v', '<', '^']
    y, x = find_position(imap)
    while not(go_out(imap, y, x)):
        nb +=1
        if nb == max_nb_visited_places:
            return True
        going_to = imap[y][x]
        dir_y, dir_x = direction[going_to]
        if imap[y + dir_y][x + dir_x] in ['#', 'O']:
            going_to = next_direction[next_direction.index(going_to) + 1]
            dir_y, dir_x = direction[going_to]
        imap[y][x] = 'X'
        y += dir_y
        x += dir_x
        imap[y][x] = going_to
    imap[y][x] = 'X'
    return False

In [14]:
def second_part(kind):
    assert kind in ('input', 'example'), '"kind" must be "input" or "example"'
    imap = read_input(kind)
    start_y, start_x = find_position(imap)
    nb_loops = 0
    for y, line in enumerate(imap):
        for x, pos in enumerate(line):
            if (y, x) == (start_y, start_x): # Don't put obstacle in starting position
                continue
            if pos == '#': # Don't put obstacle where there is already one
                continue
            # Else (must be '.'): try with obstacle:
            assert imap[y][x] == '.', f"should be '.' in position ({y}, {x}) and not '{imap[y][x]}'"
            imap[y][x] = 'O'
            if lazy_looping(imap):
                nb_loops += 1 # Count one more loop
            imap = read_input(kind) # Then come back to initial state
    return nb_loops

In [15]:
second_part("example")

6

In [16]:
second_part("input")

1697

Very long…but it works ^^